### Heroes Of Pymoli Data Analysis
* Of the 576 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

In [41]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Display top 5 rows for dataframe
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [42]:
# Finding the amount of players by looking for unique SNs
players_data = {"Total Players": [len(purchase_data["SN"].unique())]}

# Converting the data into a DataFrame
players_df = pd.DataFrame(data=players_data)

# Displaying the DataFrame
players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [43]:
# Defining the data that goes into the DataFrame
purchase_analysis = {"Number of Unique Items":[len(purchase_data["Item ID"].unique())],
                     "Average Price":[purchase_data["Price"].mean()],
                     "Number of Purchases":[len(purchase_data["Purchase ID"].unique())],
                     "Total Revenue":[purchase_data["Price"].sum()]}

# Converting data into a DataFrame
purchase_analysis_df = pd.DataFrame(data = purchase_analysis)

# Reorganizing the columns
purchase_analysis_df = purchase_analysis_df[["Number of Unique Items","Number of Purchases",
                                             "Average Price","Total Revenue"]]

# Formatting the DataFrame but still keeping the original unformatted version in case
# other mathematical functions would be used on it later
purchase_format = purchase_analysis_df.style.format({"Average Price": "${:.2f}", 
                                                     "Total Revenue": "${:.2f}"})

# Displaying the DataFrame
purchase_format

,Number of Unique Items,Number of Purchases,Average Price,Total Revenue
0,183,780,$3.05,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed Players


In [44]:
# Since some users bought multiple items, drop duplicate SN since 
# every player has a unique SN
delete_dups = purchase_data.drop_duplicates(subset=["SN"], keep="first")

# Counting total of each gender and calculating percentage
total_by_gender = {"Total Count":[delete_dups["Gender"].value_counts()[0],
                                  delete_dups["Gender"].value_counts()[1],
                                  delete_dups["Gender"].value_counts()[2]],
                   "Percentage":[delete_dups["Gender"].value_counts()[0]/len(purchase_data["SN"].unique()),
                                 delete_dups["Gender"].value_counts()[1]/len(purchase_data["SN"].unique()),
                                 delete_dups["Gender"].value_counts()[2]/len(purchase_data["SN"].unique())]}

# Creating DataFrame
gender_df = pd.DataFrame(data = total_by_gender)

# Setting the index 
genders = ["Male", "Female", "Other / Non-Disclosed"]
gender_df.set_index([genders],inplace=True)

# Formatting percentage but saving it into a new DataFrame in case calculations are needed
# later on
gender_format = gender_df.style.format({"Percentage": "{:.2%}"})

# Displaying DataFrame
gender_format

,Percentage,Total Count
Male,84.03%,484
Female,14.06%,81
Other / Non-Disclosed,1.91%,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, etc. by gender


* For normalized purchasing, divide total purchase value by purchase count, by gender


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [45]:
# Making a copy of the original DataFrame before dropping duplicates
gender_groups = purchase_data.copy()

# Dropping duplicates to find count of each gender
gender_groups.drop_duplicates(subset=["SN"], inplace=True)

# Grouping by gender
gender_groups = gender_groups.groupby("Gender")

# Calculating mean
gender_groups_price = gender_groups["Price"].mean()

# Formatting the price into currency
female_avg_price = "${:.2f}".format(gender_groups_price[0])
male_avg_price = "${:.2f}".format(gender_groups_price[1])
other_avg_price = "${:.2f}".format(gender_groups_price[2])

# Finding the total
gender_groups_total = gender_groups["Price"].sum()

# Formatting price into currency
female_total_price = "${:,.2f}".format(gender_groups_total[0])
male_total_price = "${:,.2f}".format(gender_groups_total[1])
other_total_price = "${:,.2f}".format(gender_groups_total[2])

# Calculating normalized price
norm_females = gender_groups_total[0]/len(purchase_data[purchase_data["Gender"] == "Female"])
norm_males = gender_groups_total[1]/len(purchase_data[purchase_data["Gender"] == "Male"])
norm_others = gender_groups_total[2]/len(purchase_data[purchase_data["Gender"] == "Other / Non-Disclosed"])

# Formatting price into currency
norm_fem = "${:.2f}".format(norm_females)
norm_mal = "${:.2f}".format(norm_males)
norm_oth = "${:.2f}".format(norm_others)

In [46]:
# Storing data
gender_data = {"Purchase Count":[len(purchase_data[purchase_data["Gender"] == "Female"]), 
                                 len(purchase_data[purchase_data["Gender"] == "Male"]), 
                                 len(purchase_data[purchase_data["Gender"] == "Other / Non-Disclosed"])],
               "Avg Purchase Price":[female_avg_price,male_avg_price,other_avg_price],
               "Total Purchase Price":[female_total_price,male_total_price,other_total_price],
               "Normalized Totals":[norm_fem,norm_mal,norm_oth]}

# Storing index
genders = ["Female", "Male", "Other / Non-Disclosed"]

# Creating DataFrame
gender_df = pd.DataFrame(data=gender_data, index=genders)

# Displaying DataFrame
gender_df

,Avg Purchase Price,Normalized Totals,Purchase Count,Total Purchase Price
Female,$3.17,$2.27,113,$256.43
Male,$3.05,$2.26,652,"$1,474.70"
Other / Non-Disclosed,$3.41,$2.50,15,$37.52


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [47]:
# Establish bins for ages
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Grouping each row into a bin and displaying them into a new column
purchase_data["Age Groups"] = pd.cut(purchase_data["Age"], age_bins, labels=group_names)

# Creating a new DataFrame to store information
grouped_age = pd.DataFrame({'Total Count' : purchase_data.groupby("Age Groups").size()}).reset_index()

# Adding Percentage column
grouped_age["Percentage"] = (grouped_age["Total Count"]/grouped_age["Total Count"].sum())*100
for i in range(0,len(grouped_age["Percentage"])):
    grouped_age["Percentage"][i]="{:.2f}".format(grouped_age["Percentage"][i])

# Displaying DataFrame
grouped_age

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Age Groups,Total Count,Percentage
0,<10,23,2.95
1,10-14,28,3.59
2,15-19,136,17.44
3,20-24,365,46.79
4,25-29,101,12.95
5,30-34,73,9.36
6,35-39,41,5.26
7,40+,13,1.67


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, etc. in the table below


* Calculate Normalized Purchasing


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [48]:
# Creating DataFrame
age_analysis = pd.DataFrame({'Total Count' : purchase_data.groupby("Age Groups").size()}).reset_index()

# Grouping 
gender_grouped = purchase_data.groupby("Age Groups")

# Finding total each group spent
total_spent = gender_grouped.sum()

# Creating a new column to store total purchase
age_analysis["Total Purchase"]=""

# Filling the column
for i in range(0,len(total_spent.Price)):
    age_analysis["Total Purchase"][i]=total_spent.Price[i]

# Creating new columns for normalized and average price and filling with data
age_analysis["Normalized Price"] = age_analysis["Total Purchase"]/age_analysis["Total Count"]
age_analysis["Average Purchase Price"] = age_analysis["Total Purchase"]/age_analysis["Total Count"]

# Formatting price into currency
age_format = age_analysis.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase": "${:.2f}",
                                        "Normalized Price": "${:.2f}"})
# Displaying DataFrame
age_format

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Age Groups,Total Count,Total Purchase,Normalized Price,Average Purchase Price
0,<10,23,$77.13,$3.35,$3.35
1,10-14,28,$82.78,$2.96,$2.96
2,15-19,136,$412.89,$3.04,$3.04
3,20-24,365,$1114.06,$3.05,$3.05
4,25-29,101,$293.00,$2.90,$2.90
5,30-34,73,$214.00,$2.93,$2.93
6,35-39,41,$147.67,$3.60,$3.60
7,40+,13,$38.24,$2.94,$2.94


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [49]:
# Taking original DataFrame and grouping it by SN and saving it into a new DF
top_spenders = purchase_data.groupby("SN")

# Calculating the total of every user
top_spenders = top_spenders["Price"].sum()

# Sorting all the users by the top 5 
# Setting ascending = False because by default it will sort ascending
top_spend_5 = top_spenders.sort_values(ascending=False).head(5)

# Storing SN into a list called names
names = [top_spend_5.index.values[0],top_spend_5.index.values[1],
         top_spend_5.index.values[2],top_spend_5.index.values[3],
         top_spend_5.index.values[4]]

# Storing total spending of the top 5 users
spending = [top_spend_5[0],top_spend_5[1],top_spend_5[2],top_spend_5[3],
            top_spend_5[4]]

# Creating new list to store the amount of purchases the top spenders had 
purchase_count = []
for each in names:
    purchase_count.append(purchase_data[purchase_data['SN']==each].shape[0])

# Creating DataFrame
spendings = pd.DataFrame({"SN":names,"Purchase Count":purchase_count,"Total Purchase Value":spending})

# Reorganizing columns
spendings = spendings[["SN","Purchase Count","Total Purchase Value"]]

# Creating new column
spendings["Average Purchase Price"] = spendings["Total Purchase Value"]/spendings["Purchase Count"]

# Formatting price into currency
spendings = spendings.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:.2f}"})

# Displaying DataFrame
spendings

,SN,Purchase Count,Total Purchase Value,Average Purchase Price
0,Lisosia93,5,$18.96,$3.79
1,Idastidru52,4,$15.45,$3.86
2,Chamjask73,3,$13.83,$4.61
3,Iral74,4,$13.62,$3.40
4,Iskadarya95,3,$13.10,$4.37


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [50]:
# Creating a new DataFrame taking 3 columns from the original
popular_items = purchase_data[["Item ID","Item Name","Price"]]

# Grouping new DataFrame by Item Name
popular_items = popular_items.groupby("Item Name")

# Counting each item to sort for most popular items
popular_items = popular_items["Item Name"].count()
popular_items = popular_items.sort_values(ascending=False).head(5)

# Creating a list of popular item names 
popular_list = [popular_items.index.values[0],popular_items.index.values[1],
                popular_items.index.values[2],popular_items.index.values[3],
                popular_items.index.values[4]]

# Putting the counts of the top 5 items into a list
popular_count = [popular_items[0],popular_items[1],popular_items[2],
                 popular_items[3],popular_items[4]]

# Since there are 2 "Final Critic" items with different item IDs and prices...
# Created a new list to store the rows in which the 5 items occur in order to 
# retrieve all the information about that particular item
row_id = []
for each in popular_list:
    row_id.append(purchase_data[purchase_data['Item Name']==each]['Item ID'].index[0])

# Retrieving the item ID
item_id = []
for each in row_id:
    item_id.append(purchase_data["Item ID"][each])

# Retrieving the item price
item_price = []
for each in row_id:
    item_price.append(purchase_data["Price"][each])

# Retrieving name of item
item_name = []
for each in row_id:
    item_name.append(purchase_data["Item Name"][each])

# Creating DataFrame to display results
popular_df = pd.DataFrame({"Item ID":item_id,"Item Name":item_name,
                           "Total Count":popular_count,"Price":item_price})

# Adding a new column
popular_df["Total Purchase Value"] = popular_df["Price"] * popular_df["Total Count"]

# Formatting price to currency
pop_df_format = popular_df.style.format({"Price": "${:.2f}", "Total Purchase Value": "${:.2f}"})

# Displaying formatted DataFrame
pop_df_format

,Item ID,Item Name,Price,Total Count,Total Purchase Value
0,92,Final Critic,$4.88,13,$63.44
1,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
2,82,Nirvana,$4.90,9,$44.10
3,141,Persuasion,$3.19,9,$28.71
4,145,Fiery Glass Crusader,$4.58,9,$41.22


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [51]:
# Sorting previous DataFrame
profitable = popular_df.sort_values(by="Total Purchase Value", ascending=False)

# Formatting price to currency
profitable_format = profitable.style.format({"Price": "${:.2f}", "Total Purchase Value": "${:.2f}"})

# Displaying DataFrame
profitable_format

,Item ID,Item Name,Price,Total Count,Total Purchase Value
0,92,Final Critic,$4.88,13,$63.44
1,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
2,82,Nirvana,$4.90,9,$44.10
4,145,Fiery Glass Crusader,$4.58,9,$41.22
3,141,Persuasion,$3.19,9,$28.71
